In [1]:
!pip install git+https://github.com/kmkurn/pytorch-crf.git
!pip install vncorenlp
!pip install seqeval

print("ĐÃ CÀI ĐẶT XONG.")

  Cloning https://github.com/kmkurn/pytorch-crf.git to /tmp/pip-req-build-7fqslr9l
  Running command git clone --filter=blob:none --quiet https://github.com/kmkurn/pytorch-crf.git /tmp/pip-req-build-7fqslr9l
  Resolved https://github.com/kmkurn/pytorch-crf.git to commit 623e3402d00a2728e99d6e8486010d67c754267b
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-crf: filename=pytorch_crf-0.7.2-py3-none-any.whl size=6410 sha256=a1efe0906f36ad0374485be30a48486389d553413aba2e184a07504284603d60
  Stored in directory: /tmp/pip-ephem-wheel-cache-wxj_9ban/wheels/fd/83/cc/f11543939f8911b8dcff86e2bd54423e21f779d0938958cc7f
Successfully built pytorch-crf
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645933 sha256=1340fdfb16d1d2e28a3367ca2590cc53817475494a311702eda396deb565daf2
  Stored in directory: /root/.cache/pip/wheels/80/ad/

In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from google.colab import drive
from seqeval.metrics import classification_report, f1_score
from tqdm.notebook import tqdm
import numpy as np
import os
import json # Thêm thư viện json
from torchcrf import CRF
import random

# Kết nối Google Drive
drive.mount('/content/drive')

# Kiểm tra GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ==============================================================================
# CHUẨN BỊ DỮ LIỆU VÀ ĐỊNH NGHĨA MODEL
# ==============================================================================
DRIVE_PATH = "/content/drive/MyDrive/"
SYLLABLE_DATA_FILE = os.path.join(DRIVE_PATH, "data_syllable.txt")
WORD_DATA_FILE = os.path.join(DRIVE_PATH, "data_word.txt")

def read_conll_file(file_path):
    sentences = []
    current_tokens, current_labels = [], []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                if current_tokens: sentences.append((current_tokens, current_labels)); current_tokens, current_labels = [], []
            else:
                parts = line.split(); current_tokens.append(" ".join(parts[:-1])); current_labels.append(parts[-1])
    if current_tokens: sentences.append((current_tokens, current_labels))
    return sentences

syllable_data = read_conll_file(SYLLABLE_DATA_FILE)
word_data = read_conll_file(WORD_DATA_FILE)
print(f"Đã đọc {len(syllable_data)} câu âm tiết và {len(word_data)} câu từ.")

def build_vocab(data):
    word_to_idx = {"<PAD>": 0, "<UNK>": 1}
    char_to_idx = {"<PAD>": 0, "<UNK>": 1}
    tag_to_idx = {"<PAD>": 0}
    for sentence, tags in data:
        for word in sentence:
            if word not in word_to_idx: word_to_idx[word] = len(word_to_idx)
            for char in word:
                if char not in char_to_idx: char_to_idx[char] = len(char_to_idx)
        for tag in tags:
            if tag not in tag_to_idx: tag_to_idx[tag] = len(tag_to_idx)
    return word_to_idx, char_to_idx, tag_to_idx

class NERDataset(Dataset):
    def __init__(self, data, word_to_idx, char_to_idx, tag_to_idx):
        self.data, self.word_to_idx, self.char_to_idx, self.tag_to_idx = data, word_to_idx, char_to_idx, tag_to_idx
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        sentence, tags = self.data[idx]
        word_indices = [self.word_to_idx.get(w, 1) for w in sentence]
        tag_indices = [self.tag_to_idx.get(t, self.tag_to_idx.get("O", 0)) for t in tags]
        char_indices_per_word = [torch.tensor([self.char_to_idx.get(c, 1) for c in w], dtype=torch.long) for w in sentence]
        return torch.tensor(word_indices, dtype=torch.long), char_indices_per_word, torch.tensor(tag_indices, dtype=torch.long)

def collate_fn(batch, tag_pad_idx):
    word_indices, char_indices, tag_indices = zip(*batch)
    word_indices_padded = pad_sequence(word_indices, batch_first=True, padding_value=0)
    tag_indices_padded = pad_sequence(tag_indices, batch_first=True, padding_value=tag_pad_idx)
    lengths = torch.tensor([len(seq) for seq in word_indices])
    max_word_len = max([len(chars) for sent in char_indices for chars in sent] or [1])
    padded_chars = []
    for sent in char_indices:
        sent_padded_chars = []
        for chars in sent:
            padded = torch.zeros(max_word_len, dtype=torch.long); padded[:len(chars)] = chars; sent_padded_chars.append(padded)
        for _ in range(word_indices_padded.shape[1] - len(sent_padded_chars)): sent_padded_chars.append(torch.zeros(max_word_len, dtype=torch.long))
        padded_chars.append(torch.stack(sent_padded_chars))
    return word_indices_padded, torch.stack(padded_chars), tag_indices_padded, lengths

class BiLSTM_CNN_CRF(nn.Module):
    def __init__(self, vocab_size, char_vocab_size, tag_to_idx, word_emb_dim, char_emb_dim, char_cnn_out_channels, hidden_dim, num_lstm_layers, dropout):
        super(BiLSTM_CNN_CRF, self).__init__()
        self.tagset_size = len(tag_to_idx)
        self.word_embedding = nn.Embedding(vocab_size, word_emb_dim, padding_idx=0)
        self.char_embedding = nn.Embedding(char_vocab_size, char_emb_dim, padding_idx=0)
        self.char_cnn = nn.Conv1d(char_emb_dim, char_cnn_out_channels, kernel_size=3, padding=1)
        self.bilstm = nn.LSTM(word_emb_dim + char_cnn_out_channels, hidden_dim // 2, num_layers=num_lstm_layers, bidirectional=True, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        self.crf = CRF(self.tagset_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
    def forward(self, words, chars, tags=None, mask=None):
        word_embeds = self.word_embedding(words)
        batch_size, seq_len, word_len = chars.shape
        chars = chars.view(batch_size * seq_len, word_len)
        char_embeds = self.char_embedding(chars).transpose(1, 2)
        char_cnn_out = self.char_cnn(char_embeds)
        char_features = torch.max(char_cnn_out, 2)[0].view(batch_size, seq_len, -1)
        combined_embeds = self.dropout(torch.cat((word_embeds, char_features), 2))
        lstm_out, _ = self.bilstm(combined_embeds)
        emissions = self.hidden2tag(self.dropout(lstm_out))
        if tags is not None: return -self.crf(emissions, tags, mask=mask.byte(), reduction='mean')
        else: return self.crf.decode(emissions, mask=mask.byte())

# ==============================================================================
# HÀM HUẤN LUYỆN
# ==============================================================================
def train_and_evaluate_bilstm_crf(train_data, val_data, test_data,
                                  word_to_idx, char_to_idx, tag_to_idx, idx_to_tag,
                                  data_type_name, output_dir_base):
    print(f"\n--- Bắt đầu xử lý cho BiLSTM-CNN-CRF với dữ liệu: {data_type_name} ---")
    WORD_EMB_DIM, CHAR_EMB_DIM, CHAR_CNN_OUT, HIDDEN_DIM = 100, 30, 30, 256
    NUM_LSTM_LAYERS, DROPOUT, BATCH_SIZE, LEARNING_RATE, EPOCHS = 2, 0.5, 16, 0.001, 20
    tag_pad_idx = tag_to_idx["<PAD>"]

    train_loader = DataLoader(NERDataset(train_data, word_to_idx, char_to_idx, tag_to_idx), batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda b: collate_fn(b, tag_pad_idx))
    val_loader = DataLoader(NERDataset(val_data, word_to_idx, char_to_idx, tag_to_idx), batch_size=BATCH_SIZE, collate_fn=lambda b: collate_fn(b, tag_pad_idx))
    test_loader = DataLoader(NERDataset(test_data, word_to_idx, char_to_idx, tag_to_idx), batch_size=BATCH_SIZE, collate_fn=lambda b: collate_fn(b, tag_pad_idx))

    model = BiLSTM_CNN_CRF(len(word_to_idx), len(char_to_idx), tag_to_idx, WORD_EMB_DIM, CHAR_EMB_DIM, CHAR_CNN_OUT, HIDDEN_DIM, NUM_LSTM_LAYERS, DROPOUT).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    best_val_f1 = -1
    output_dir = os.path.join(output_dir_base, f"bilstm-cnn-crf-{data_type_name}")
    os.makedirs(output_dir, exist_ok=True)
    best_model_path = os.path.join(output_dir, "best_model.pt")

    # ===== Định nghĩa đường dẫn cho file từ điển =====
    vocabs_path = os.path.join(output_dir, "vocabs.json")

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for words, chars, tags, lengths in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Training]"):
            words, chars, tags, lengths = words.to(device), chars.to(device), tags.to(device), lengths.to(device)
            mask = torch.arange(words.size(1), device=device)[None, :] < lengths[:, None]
            loss = model(words, chars, tags, mask)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_loss += loss.item()

        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for words, chars, tags, lengths in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Validation]"):
                words, chars, lengths = words.to(device), chars.to(device), lengths.to(device)
                mask = torch.arange(words.size(1), device=device)[None, :] < lengths[:, None]
                preds = model(words, chars, mask=mask)
                for i, length in enumerate(lengths):
                    all_preds.append([idx_to_tag.get(p, "O") for p in preds[i]])
                    all_labels.append([idx_to_tag.get(t, "O") for t in tags[i][:length].tolist()])

        val_f1 = f1_score(all_labels, all_preds)
        print(f"Epoch {epoch+1}, Train Loss: {total_loss/len(train_loader):.4f}, Validation F1: {val_f1:.4f}")

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model saved with F1: {val_f1:.4f}")

            # ===== Lưu các file từ điển CÙNG LÚC với model tốt nhất =====
            vocabs_to_save = {
                "word_to_idx": word_to_idx,
                "char_to_idx": char_to_idx,
                "tag_to_idx": tag_to_idx
            }
            with open(vocabs_path, "w", encoding="utf-8") as f:
                json.dump(vocabs_to_save, f, ensure_ascii=False, indent=4)
            print(f"Vocabularies saved to {vocabs_path}")


    print("\n--- Testing with the best model ---")
    model.load_state_dict(torch.load(best_model_path))
    model.eval()
    all_preds_test, all_labels_test = [], []
    with torch.no_grad():
        for words, chars, tags, lengths in tqdm(test_loader, desc="[Testing]"):
            words, chars, lengths = words.to(device), chars.to(device), lengths.to(device)
            mask = torch.arange(words.size(1), device=device)[None, :] < lengths[:, None]
            preds = model(words, chars, mask=mask)
            for i, length in enumerate(lengths):
                all_preds_test.append([idx_to_tag.get(p, "O") for p in preds[i]])
                all_labels_test.append([idx_to_tag.get(t, "O") for t in tags[i][:length].tolist()])
    report = classification_report(all_labels_test, all_preds_test, digits=4, zero_division=0)
    print("\nBáo cáo chi tiết trên tập TEST:")
    print(report)

    return {"model": "BiLSTM-CNN-CRF", "data_type": data_type_name,
            "test_f1": f1_score(all_labels_test, all_preds_test),
            "classification_report_test": report, "best_model_path": best_model_path}

# ==============================================================================
# CHẠY
# ==============================================================================
random.seed(42)
if syllable_data: random.shuffle(syllable_data)
if word_data: random.shuffle(word_data)
train_size = int(0.7 * len(syllable_data))
val_size = int(0.15 * len(syllable_data))
train_syllable, val_syllable, test_syllable = syllable_data[:train_size], syllable_data[train_size:train_size+val_size], syllable_data[train_size+val_size:]
train_word, val_word, test_word = word_data[:train_size], word_data[train_size:train_size+val_size], word_data[train_size+val_size:]
word_to_idx_s, char_to_idx_s, tag_to_idx_s = build_vocab(train_syllable)
idx_to_tag_s = {i:t for t,i in tag_to_idx_s.items()}
word_to_idx_w, char_to_idx_w, tag_to_idx_w = build_vocab(train_word)
idx_to_tag_w = {i:t for t,i in tag_to_idx_w.items()}
ALL_OUTPUTS_DIR = os.path.join(DRIVE_PATH, "kpi_ner_outputs_bilstm_with_vocabs")
os.makedirs(ALL_OUTPUTS_DIR, exist_ok=True)
results_bilstm = []
result_syllable = train_and_evaluate_bilstm_crf(train_syllable, val_syllable, test_syllable, word_to_idx_s, char_to_idx_s, tag_to_idx_s, idx_to_tag_s, "syllable", ALL_OUTPUTS_DIR)
results_bilstm.append(result_syllable)
result_word = train_and_evaluate_bilstm_crf(train_word, val_word, test_word, word_to_idx_w, char_to_idx_w, tag_to_idx_w, idx_to_tag_w, "word", ALL_OUTPUTS_DIR)
results_bilstm.append(result_word)
print("\n\n--- TỔNG HỢP KẾT QUẢ BiLSTM-CNN-CRF ---")
for res in results_bilstm:
    print(f"\nKết quả cho mô hình {res['model']} với dữ liệu {res['data_type']}:")
    print(f"  Test F1-score: {res['test_f1']:.4f}")
    print(f"  Đường dẫn mô hình: {res['best_model_path']}")
    print("  Báo cáo chi tiết trên tập Test:")
    print(res['classification_report_test'])

Mounted at /content/drive
Using device: cuda
Đã đọc 2500 câu âm tiết và 2500 câu từ.

--- Bắt đầu xử lý cho BiLSTM-CNN-CRF với dữ liệu: syllable ---


Epoch 1/20 [Training]:   0%|          | 0/110 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torchcrf/__init__.py:252: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at /pytorch/aten/src/ATen/native/TensorCompare.cpp:529.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Epoch 1/20 [Validation]:   0%|          | 0/24 [00:00<?, ?it/s]

Epoch 1, Train Loss: 38.0225, Validation F1: 0.6165
New best model saved with F1: 0.6165
Vocabularies saved to /content/drive/MyDrive/kpi_ner_outputs_bilstm_with_vocabs/bilstm-cnn-crf-syllable/vocabs.json


...

Epoch 20/20 [Training]:   0%|          | 0/110 [00:00<?, ?it/s]

Epoch 20/20 [Validation]:   0%|          | 0/24 [00:00<?, ?it/s]

Epoch 20, Train Loss: 0.2599, Validation F1: 0.9804

--- Testing with the best model ---


[Testing]:   0%|          | 0/24 [00:00<?, ?it/s]


Báo cáo chi tiết trên tập TEST:
              precision    recall  f1-score   support

   DIRECTION     0.9886    0.9858    0.9872       351
         KPI     0.9734    0.9760    0.9747       375
       OWNER     0.9973    0.9973    0.9973       365
      TARGET     0.9812    0.9759    0.9786       374
  TIME_FRAME     0.9917    0.9917    0.9917       360
        UNIT     0.9801    0.9801    0.9801       352

   micro avg     0.9853    0.9844    0.9848      2177
   macro avg     0.9854    0.9845    0.9849      2177
weighted avg     0.9853    0.9844    0.9848      2177



--- TỔNG HỢP KẾT QUẢ BiLSTM-CNN-CRF ---

Kết quả cho mô hình BiLSTM-CNN-CRF với dữ liệu syllable:
  Test F1-score: 0.9921
  Đường dẫn mô hình: /content/drive/MyDrive/kpi_ner_outputs_bilstm_with_vocabs/bilstm-cnn-crf-syllable/best_model.pt
  Báo cáo chi tiết trên tập Test:
              precision    recall  f1-score   support

   DIRECTION     0.9824    0.9767    0.9795       343
         KPI     0.9973    0.9973    0.9